In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np



In [2]:
def calculate_transfer_function(params, freqs):
    """
    Calculates the Power Spectral Density (PSD) of the current-dipole moment
    for a ball-and-two-sticks neuron model with a somatic white noise current input.

    Args:
        params (dict): A dictionary containing the biophysical parameters.
        freqs (np.ndarray): An array of frequencies (in Hz) to calculate the PSD for.

    Returns:
        tuple: A tuple containing:
            - freqs (np.ndarray): The input frequency array.
            - S_p (np.ndarray): The calculated PSD of the dipole moment.
    """
    # --- 1. Extract parameters and convert to SI units ---
    Rm = params['Rm']  # Specific membrane resistance (Ohm * m^2)
    Ri = params['Ri']  # Axial resistivity (Ohm * m)
    Cm = params['Cm']  # Specific membrane capacitance (F / m^2)
    d = params['d']    # Stick diameter (m)
    ds = params['ds']  # Soma diameter (m)
    l1 = params['l1']  # Length of stick 1 (m)
    l2 = params['l2']  # Length of stick 2 (m)
    s_in = params['s_in'] # PSD of white noise input current (A^2 / Hz)

    # --- 2. Calculate derived cable properties ---
    # Angular frequency
    omega = 2 * np.pi * freqs
    # Add a small epsilon to the first frequency to avoid division by zero if f=0
    omega[0] = omega[0] + 1e-9

    # Membrane time constant (s)
    tau_m = Rm * Cm

    # Length constant (m)
    # This is the standard definition from cable theory (e.g., Rall, 1959)
    lambda_ = np.sqrt((d * Rm) / (4 * Ri))

    # Electrotonic lengths (dimensionless)
    L1 = l1 / lambda_
    L2 = l2 / lambda_

    # Dimensionless frequency
    W = omega * tau_m

    # Complex frequency-dependent term q
    q = np.sqrt(1 + 1j * W)

    # --- 3. Calculate admittances (inverse of impedance) ---
    # Admittance of the soma (S)
    # Ys = Area * (conductance + j*omega*capacitance)
    #Ys = np.pi * ds**2 * (1/Rm + 1j * omega * Cm)
    Ys = np.pi * ds**2 * (1/Rm) * q**2

    # Infinite-stick admittance (S)
    # This is the admittance of a semi-infinite cable.
    #Y_inf = (np.pi * d**1.5 * np.sqrt(1/Rm)) / (2 * np.sqrt(Ri)) * q
    G_inf = np.pi * d**2 / (4 * Ri * lambda_)
    Y_inf = q * G_inf

    # Total input admittance seen by the soma (S)
    # This is the parallel sum of the soma and the two finite sticks.
    Y_in_total = Ys + Y_inf * (np.tanh(q * L1) + np.tanh(q * L2))

    # --- 4. Calculate the transfer function T_p(f) ---
    # This is the function that converts input current to dipole moment.
    # It is derived from first principles for the two-stick model.
    # The numerator represents the difference in dipole moments from the two sticks.
    numerator = lambda_ * G_inf * (1/np.cosh(q * L2) - 1/np.cosh(q * L1))

    # The denominator is the total load (admittance) the current source sees.
    denominator = Y_in_total

    T_p = numerator / denominator
    abs_T_p = np.abs(T_p)
    abs_T_p_squared = np.abs(T_p)**2

    # --- 5. Calculate the output PSD ---
    # S_out(f) = |T(f)|^2 * S_in(f)
    # For white noise, S_in(f) is constant.
    S_p = np.abs(T_p)**2 * s_in

    return freqs, S_p, abs_T_p